# Start Here Notebook


This notebook is used to make sure that your setup is ready to run for the workshop. 
The main steps are
* Set constants that are used throughout the workshop, e.g. `project_prefix`, `bucket_prefix`, etc
* Set Mlflow server `mlflow_arn` that will be used throughout the workshop
* Install Docker package to enable Studio local mode

It also provides you instructions how to resolve issues, in case you are running this on your own AWS account.

To run this notebook and all notebooks in the workshop please use the `Python 3` kernel in JupyterLab

## Star GitHub repository

In [ ]:
%%html

<a class="github-button" href="https://github.com/aws-samples/amazon-sagemaker-build-train-deploy" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star Amazon SageMaker secure MLOps on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>

### Click this button ^^^ above ^^^

## Setup
Get the latest version of SageMaker Python SDK.

<div class="alert alert-info"> 💡 The workshop and all notebooks were tested with Sagemaker Distribution `1.10` and the SageMaker Python SDK (the package sagemaker) version 2.219.0. The notebooks don't pin the version of the sagemaker. If you encounter any incompatibility issues, you can install the specific version of the sagemaker by running the pip command: <code>%pip install sagemaker=2.219.0</code>
</div>

In [ ]:
# Uncomment if you have any compatibility issues and would like to use the specific version of the sagemaker library
# %pip install sagemaker==2.219.0
#%pip install --upgrade pip sagemaker boto3

In [ ]:
#%pip install mlflow==2.13.2 sagemaker-mlflow

### Import packages

In [ ]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep

(sagemaker.__version__,boto3.__version__)

### Set constants

In [ ]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
project_prefix = "amzn"
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = f"{bucket_name}/{project_prefix}"
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()

initialized = True

print(sm_role)

In [ ]:
# Store some variables to keep the value between the notebooks
%store bucket_name
%store project_prefix   
%store bucket_prefix
%store sm_role
%store region
%store initialized

### Get domain id
You need this value `domain_id` in many SageMaker Python SDK and boto3 SageMaker API calls. The notebook metadata file contains `domain_id` value. The following code demonstrates how to access the notebook metadata file and get the `domain_id`.

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"User profile: {user_profile_name}")

%store domain_id
%store space_name
%store user_profile_name

### Connect to MLflow tracking server
If you're running an AWS-led workshop or used the delivered CloudFormation template to provision your workshop environment, an MLflow server must be up and running. If you don't have an MLflow server, follow the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html) or run the following code cell to create a new one.

To create and manage an MLflow tracking server and to work with managed MLflow experiements, you need the following permissions attached to the SageMaker execution role:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],
            "Resource": "*"
        }
    ]
}
```

Execute the following code to check if you have a running MLflow server.

In [ ]:
# Find an active MLflow server in the account
r = boto3.client("sagemaker").list_mlflow_tracking_servers(
    TrackingServerStatus='Created',
)['TrackingServerSummaries']

if len(r) < 1:
    print("You don't have any running MLflow servers. Trying to find a server in the status 'Creating'...")

    r = boto3.client("sagemaker").list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("You don't have any MLflow server in the status 'Creating'. Run the next code cell to create a new one.")
        mlflow_arn = None
        mlflow_name = None
    else:
        mlflow_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"You have an MLflow server {mlflow_arn} in the status 'Creating', going to use this one")
else:
    mlflow_arn = r[0]['TrackingServerArn']
    mlflow_name = r[0]['TrackingServerName']
    print(f"You have {len(r)} running MLflow server(s). Get the first server ARN:{mlflow_arn}")

In [ ]:
# This code cell creates a new MLflow server
if not mlflow_arn:
    ts = strftime('%d-%H-%M-%S', gmtime())
    mlflow_name = f"mlflow-{domain_id}-{ts}"
    r = boto3.client("sagemaker").create_mlflow_tracking_server(
        TrackingServerName=mlflow_name,
        ArtifactStoreUri=f"s3://{bucket_name}/mlflow/{ts}",
        RoleArn=sm_role,
        AutomaticModelRegistration=True,
    )

    mlflow_arn = r['TrackingServerArn']
    print(f"Server creation request succeded. The server {mlflow_arn} is being created.")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
Creation of an MLflow server can take up to 25 minutes. You don't need to wait - proceed with the flow of the workshop.
</div>

In [ ]:
(mlflow_arn, mlflow_name)

In [ ]:
%store mlflow_arn
%store mlflow_name

## Install Docker to enable Studio local mode
Amazon SageMaker Studio applications support the use of local mode to create estimators, processors, and pipelines, then deploy them to a local environment. With local mode, you can test machine learning scripts before running them in Amazon SageMaker managed training or hosting environments. Refer to [Local mode support in Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html) to understand which docker operations the Studio currently supports.

To use local mode in Studio applications, you must install Docker into your JupyterLab space. 

### Check if docker access is enabled

In [ ]:
# check that docker enabled in the SageMaker domain
docker_settings = sm_client.describe_domain(DomainId=domain_id)['DomainSettings'].get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"The docker access is ENABLED in the domain {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"You must enable docker access in the domain to use Studio local mode")

<div style="border: 4px solid coral; text-align: center; margin: auto;">
If the previous code cell raised an exeption that the docker access is not enabled, you need to enable the access. See the following instructions how to do it.
</div>

In [ ]:
print(f"Domain id: {domain_id}")

### Enable docker access for the SageMaker domain

<div class="alert alert-info">You only need this section if the docker access is not enabled in the domain.
</div>

You need `sagemaker:UpdateDomain` permission in the execution role. You cannot update domain from this notebook because the notebook execution role doesn't have this permission. To update domain settings, you can use one of the following options.

By adding `'VpcOnlyTrustedAccounts': ['885854791233']`, we will allow docker within Sagemaker Studio to also pull images from the sagemaker-distribution repo, which would otherwise be restricted if you are using Sagemaker in VPC-Only Mode.

#### Option 1: run `update_domain` in the notebook
If you have the corresponding permissions in the notebook execution role, you can run the following code in a notebook:

```python
import boto3

r = boto3.client('sagemaker').update_domain(
    DomainId=domain_id,
    DomainSettingsForUpdate={
        'DockerSettings': {
            'EnableDockerAccess':'ENABLED',
            'VpcOnlyTrustedAccounts': ['885854791233'] # us-east-1 account, to be changed for other regions, see below
        }
    }
)
```

#### Option 2: run `aws sagemaker` CLI in the  terminal
Make sure you run `AWS CLI` in the terminal where you have the corresponding permissions `sagemaker:UpdateDomain`. Run the following command, e.g. for us-east-1:

```
 aws sagemaker update-domain --domain-id <DOMAIN-ID> --domain-settings-for-update '{"DockerSettings": {"EnableDockerAccess": "ENABLED","VpcOnlyTrustedAccounts": ["885854791233"]}}'
```

For example, you can run the command above in the [AWS CloudShell](https://aws.amazon.com/blogs/aws/aws-cloudshell-command-line-access-to-aws-resources/) in your AWS account.

For other regions make sure to adjust the account id as described below.

#### Determining the ECR Repo image for VPC only Mode

We will determine the public ECR repository for the sagemaker distribution.
Make sure that your role has access to get the parameter over ssm. The policy would look like this

```
{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Sid": "GetECRRepoAccountID",
			"Effect": "Allow",
			"Action": [
				"ssm:GetParameter"
			],
			"Resource": [
				"arn:aws:ssm:*::parameter/aws/service/sagemaker-distribution/ecr-account-id"
			]
		}
	]
}
```

In [ ]:
import boto3

REGION = boto3.session.Session().region_name
REGION = "us-east-1"
sagemaker_dist_repos = "/aws/service/sagemaker-distribution/ecr-account-id"
sm_dist_repo_account = boto3.client('ssm', region_name=REGION).get_parameter(Name=sagemaker_dist_repos)['Parameter']['Value']

SM_DIST_IMAGE=f"{sm_dist_repo_account}.dkr.ecr.{REGION}.amazonaws.com/sagemaker-distribution-prod:1.11.0-gpu"
print(f"Sagemaker distribution account for region: {REGION}: {sm_dist_repo_account}")
print(f"SM_DIST_IMAGE: {SM_DIST_IMAGE}")

In [ ]:
# check the updated settings
sm_client.describe_domain(DomainId=domain_id)['DomainSettings']

### Install Docker

In [ ]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Currently only Docker version 20.10.X is supported in Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validate the Docker Client is able to access Docker Server at [unix:///docker/proxy.sock]
docker version

## Restart kernel

In [ ]:
# Restart kernel to get the packages
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## Further workshop flow
Continue with the workshop by going to Module 01.
